In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import os

In [ ]:
def xml_findvalue (instance, name):
    if instance.find(".//%s" % name) == None:
        value = None
    else:
        value = instance.find(".//%s" % name).text
    return value

abstract_values = ['PMID', 'ISSN', 'Volume', 'Issue', 'Year', 'Month', 'Title', 'ArticleTitle', 'AbstractText', 'Affiliation','PublicationType']

author_values = ['LastName', 'ForeName', 'Initials']
author_values_PMID = ['PMID','LastName', 'ForeName', 'Initials']


mesh_values = ['DescriptorName', 'QualifierName']
mesh_values_PMID = ['PMID', 'MajorTopicYN', 'DescriptorName', 'QualifierName']

In [ ]:
roots = []
directory = '/Users/jenniferpolson/Documents/School/PhD/Academic/2018-S/BE 224B/Midterm Project/XML Parsing'
for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        tree = ET.parse(filename)
        root = tree.getroot()
        roots.append(root)

In [ ]:
df_abstract = []
df_authors = []
df_mesh = []

In [ ]:
for r in roots:
    for k in np.arange(0,len(r),1):
    #for k in np.arange(0,10,1):
        #look at the article in question
        article = r.getchildren()[k]
        #generate the abstract text entry
        language = xml_findvalue (article, 'Language')
        if language == 'eng':
            entry = []
            for i in abstract_values:
                value = xml_findvalue (article, i)
                entry.append(value)
            #df_abstract = df_abstract.append(pd.Series(entry, index=abstract_values) ,ignore_index=True)
            df_abstract.append(entry)

            #generate ALL of the authors
            authors = article.findall(".//%s" % 'Author')
            for f in np.arange(0,len(authors),1):
                name_entry = []
                name_entry.append(article.find('.//%s' % 'PMID').text)
                for i in author_values:
                    value = xml_findvalue (authors[f], i)
                    name_entry.append(value)
                #df_authors = df_authors.append(pd.Series(name_entry, index=author_values_PMID) ,ignore_index=True)
                df_authors.append(name_entry)

            #generate ALL of the MeSH terms
            mesh_terms = article.findall(".//%s" % 'MeshHeading')
            for f in np.arange(0,len(mesh_terms),1):
                name_entry = []
                name_entry.append(article.find('.//%s' % 'PMID').text)
                name_entry.append(article.find('.//%s' % 'DescriptorName').attrib['MajorTopicYN'])
                for i in mesh_values:
                    value = xml_findvalue (mesh_terms[f], i)
                    name_entry.append(value)
                #df_mesh = df_mesh.append(pd.Series(name_entry, index=mesh_values_PMID) ,ignore_index=True)
                df_mesh.append(name_entry)
        else:
            language = language

In [ ]:
df_abstract = pd.DataFrame(df_abstract, columns = abstract_values)
df_authors = pd.DataFrame(df_authors, columns = author_values_PMID)
df_mesh = pd.DataFrame(df_mesh, columns = mesh_values_PMID)

In [ ]:
df_abstract.to_csv("abstract_table.csv",index=False)
df_authors.to_csv("author_table.csv", index = False)
df_mesh.to_csv("mesh_table.csv", index = False)